### Constructing an Agent-Based Pipeline Integrated with RAG-based Application with Various Data Sources and Tools

In [ ]:
!pip install -q llama-index deeplake==3.9.26 openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.7/617.7 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.5 MB/s eta 0:00:00

In [ ]:
!pip install llama-index-agent-openai

In [ ]:
import os
import openai
import getpass

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


In [ ]:
os.environ["ACTIVELOOP_TOKEN"] = getpass.getpass("Activeloop Token:")

Activeloop Token:··········


In [ ]:
#defining data sources
!mkdir -p 'data/1k/'
!wget 'https://github.com/idontcalculate/data-repo/blob/main/machine_to_end_war.txt' -O './data/1k/tesla.txt'
!wget 'https://github.com/idontcalculate/data-repo/blob/main/prodigal_chapter10.txt' -O './data/1k/web.txt'

--2024-11-26 05:52:37--  https://github.com/idontcalculate/data-repo/blob/main/machine_to_end_war.txt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘./data/1k/tesla.txt’

./data/1k/tesla.txt     [ <=>                ] 176.77K  1010KB/s    in 0.2s    

2024-11-26 05:52:38 (1010 KB/s) - ‘./data/1k/tesla.txt’ saved [181017]

--2024-11-26 05:52:38--  https://github.com/idontcalculate/data-repo/blob/main/prodigal_chapter10.txt
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘./data/1k/web.txt’

./data/1k/web.txt       [ <=>                ] 186.14K  1.00MB/s    in 0.2s    

2024-11-26 05:52:38 (1.00 MB/s) - ‘./data/1k/web.txt’ saved [190609]



In [ ]:
from llama_index.core import SimpleDirectoryReader

tesla_docs = SimpleDirectoryReader( input_files=["/content/data/1k/tesla.txt"] ).load_data()

In [ ]:
tesla_docs

[Document(id_='7a7f2c81-5f62-4553-8270-21f91916214d', embedding=None, metadata={'file_path': '/content/data/1k/tesla.txt', 'file_name': 'tesla.txt', 'file_type': 'text/plain', 'file_size': 181017, 'creation_date': '2024-11-26', 'last_modified_date': '2024-11-26'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='\n\n\n\n\n\n<!DOCTYPE html>\n<html\n  lang="en"\n  \n  data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"\n  data-a11y-animated-images="system" data-a11y-link-underlines="true"\n  \n  >\n\n\n\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com"

In [ ]:
!pip install llama-index-vector-stores-deeplake

In [ ]:
!pip install llama-index-embeddings-openai

In [ ]:
#srore indexes in deep lake
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.deeplake import DeepLakeVectorStore

embeddings = OpenAIEmbedding(model = "text-embedding-ada-002")

my_activeloop_org_id = "jacobasir"
my_activeloop_dataset_name = "tesla_predictions_with_llamaindex"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

# Create an index over the documnts
vector_store = DeepLakeVectorStore(dataset_path=dataset_path,embedding_function=embeddings, overwrite=False)

/usr/local/lib/python3.10/dist-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (4.0.3) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


Your Deep Lake dataset has been successfully created!


In [ ]:
#utilizing db object to create storage context and it allows to generate indexes(embeddings) and save them into db using VectorStoreIndex class
from llama_index.core import VectorStoreIndex

from llama_index.core import StorageContext

storage_context = StorageContext.from_defaults(vector_store=vector_store)

tesla_index = VectorStoreIndex.from_documents(tesla_docs, storage_context=storage_context)

Uploading data to deeplake dataset.


100%|██████████| 86/86 [00:01<00:00, 64.96it/s]
-

Dataset(path='hub://jacobasir/tesla_predictions_with_llamaindex', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (86, 1)      str     None   
 metadata     json      (86, 1)      str     None   
 embedding  embedding  (86, 1536)  float32   None   
    id        text      (86, 1)      str     None   


above are the indexes for the first file and ready to be used as source.

In [ ]:
webtext_docs = SimpleDirectoryReader( input_files=["/content/data/1k/web.txt"] ).load_data()

In [ ]:
webtext_docs

[Document(id_='1765ef14-764d-4dbe-bd11-01377ee7e432', embedding=None, metadata={'file_path': '/content/data/1k/web.txt', 'file_name': 'web.txt', 'file_type': 'text/plain', 'file_size': 190609, 'creation_date': '2024-11-26', 'last_modified_date': '2024-11-26'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='\n\n\n\n\n\n<!DOCTYPE html>\n<html\n  lang="en"\n  \n  data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"\n  data-a11y-animated-images="system" data-a11y-link-underlines="true"\n  \n  >\n\n\n\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">\n 

Storing index locally

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

try:
  # Try to load the index if it is already calculated
  storage_context = StorageContext.from_defaults( persist_dir="/content/storage/webtext" )
  webtext_index = load_index_from_storage(storage_context)
  print("Loaded the pre-computed index.")
except:
  # Otherwise, generate the indexes
  webtext_index = VectorStoreIndex.from_documents(webtext_docs)
  webtext_index.storage_context.persist(persist_dir="/content/storage/webtext") #if indexes are not calculated, it stores indexes using .persist() method.
  print("Generated the index.")

Generated the index.


### Query Engine

Query engine for creating an agent which is capable of integrating info from two sources is easy.

In [ ]:
tesla_engine = tesla_index.as_query_engine(similarity_top_k = 3)
webtext_engine = webtext_index.as_query_engine(similarity_top_k = 3)

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=tesla_engine,
        metadata=ToolMetadata(
            name="tesla_1k",
            description=(
                "Provides information about Tesla's statements that refers to future times and predictions. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=webtext_engine,
        metadata=ToolMetadata(
            name="webtext_1k",
            description=(
                "Provides information about tesla's life and biographical data. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

### Agent

In [ ]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

agent = OpenAIAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

In [ ]:
agent.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.

Human: What influenced Nikola Tesla to become an inventor?
Added user message to memory: What influenced Nikola Tesla to become an inventor?
=== Calling Function ===
Calling function: webtext_1k with args: {"input":"What influenced Nikola Tesla to become an inventor?"}
Got output: The experiences and observations Nikola Tesla had with mechanical vibrations, particularly through his experiments with various oscillators and their effects on physical objects, influenced him to become an inventor.

Assistant: Nikola Tesla was influenced to become an inventor by his experiences and observations with mechanical vibrations, particularly through his experiments with various oscillators and their effects on physical objects.

Human: Who was Nikola Tesla ?
Added user message to memory: Who was Nikola Tesla ?
=== Calling Function ===
Calling function: webtext_1k with args: {"input":"Who was Nikola Tesla?"}
Got output: Nikola Tesla was a brillia